# Assignment 2
In this assignment, we will go through Perceptron, Linear Classifiers, Loss Functions, Gradient Descent and Back Propagation.


PS. this one is not from Stanford's course.



\

## Instructions
* This notebook contain blocks of code, you are required to complete those blocks(where required)
* You are required to copy this notebook ("copy to drive" above) and complete the code.(DO NOT CHANGE THE NAME OF THE FUNCTIONS)

# Part 1: Perceptron
In this section, we will see how to implement a perceptron. Goal would be for you to delve into the mathematics.


## Intro
What's a perceptron? It's an algorithm modelled on biological computational model to classify things into binary classes. It's a supervides learning algorithm, meaning that you need to provide labelled data containing features and the actual classifications. A perceptron would take these features as input and spit out a binary value (0 or 1). While training the model with training data, we try to minimise the error and learn the parameters involved.

**How does it work?**\
A perceptron is modelled on a biological neuron. A neuron has input dendrites and the output is carried by axons. Similarly, a perceptron takes inputs called "features". After processing, a perceptron gives output. For computation, it has a "weight" vector which is multipled with feature vector. An activation function is added to introduce some non linearities and the output is given out.\
It can be represented as: $$  f=\sum_{i=1}^{m} w_ix_i +b$$

Let's implement this simple function to give an output.



In [1]:
import numpy as np

class perceptron():
    def __init__(self,num_input_features=8):
        self.weights = np.random.randn(num_input_features)
        self.bias = np.random.random()

    def activation(self,x):
        '''
            Implement heavside step activation function here (google ;))
        '''
        if x >= 0:
          return 1
        else:
          return 0


    def forward(self,x: np.ndarray):
        '''
            you have random initialized weights and bias
            you can access then using `self.weights` and `self.bias`
            you should use activation function before returning
        
            x : input features
            return : a binary value as the output of the perceptron 
        '''
        w = self.weights  
        b = self. bias 
        f = x.dot(w) + b
        return self.activation(f)

In [2]:
np.random.seed(0)
perc = perceptron(8)
assert perc.forward(np.arange(8))==1

# Part 2: Linear Classifier
In this section, we will see how to implement a linear Classifier.


## Intro


**How does it work?**

Linear Classifier uses the following function: $$Y = WX+b$$ Where, $W$ is a 2d array of weights with shape (#classes, #features).



Let's implement this classifier.



In [3]:
import numpy as np

class LinearClassifier():
    def __init__(self,num_input_features=32,num_classes=5):
        self.weights = np.random.randn(num_classes,num_input_features)
        self.bias = np.random.rand(num_classes,1)

    def forward(self,x: np.ndarray):
        '''
            x: input features
            you have random initialized weights and bias
            you can access then using `self.weights` and `self.bias`
            return an output vector of num_classes size
        '''
        return self.weights.dot(x) + self.bias
        pass

In [4]:
np.random.seed(0)
lc = LinearClassifier()
lc.forward(np.random.rand(32,1))
# Should be close to:
# array([[  7.07730669],
    #    [-10.24067722],
    #    [  0.75398702],
    #    [  9.8019519 ],
    #    [  2.36684038]])

array([[  7.07730669],
       [-10.24067722],
       [  0.75398702],
       [  9.8019519 ],
       [  2.36684038]])

# Part 3: Loss Functions, Gradient descent and Backpropagation




## Intro

Loss Functions tells how "off" the output od our model is. Based upon the application, you can use several different loss functions. Formally, A loss function is a function $L:(z,y)\in\mathbb{R}\times Y\longmapsto L(z,y)\in\mathbb{R}$ that takes as inputs the predicted value $z$ corresponding to the real data value yy and outputs how different they are We'll implement L1 loss, L2 loss, Logistic loss, hinge loss and cross entropy loss functions.

### **L1 loss**
L1 loss is the linear loss function  $L = \dfrac{1}{2}|y−z| $



In [5]:
import numpy as np
def L1loss(z,y):
    '''
        y : True output.
        z : Predicted output.
        return : L
    '''
    return np.sum(0.5*abs(y-z))
    

### **L2 loss**
L2 loss is the quadratic loss function or the least square error function  $L = \dfrac{1}{2}(y−z)^2 $



In [6]:
import numpy as np
def L2loss(z,y):
    '''
        y : True output. 
        z : Predicted output. 
        return : L
    '''
    return np.sum(0.5*((y-z)**2))
    

### **Hinge Loss**
Hinge loss is: $ L = max( 0, 1 - yz ) $

In [7]:
import numpy as np
def hingeLoss(z,y):
    '''
        y : True output. 
        z : Predicted output. 
        return : L
    '''
    L = 0
    for i in range(len(y)):
      L += max(0,1-y[i]*z[i])
    return L
    

### **Cross Entropy Loss**
Another very famous loss function is Cross Entropy loss: $ L = −[ylog(z)+(1−y)log(1−z)] $.

In [8]:
import numpy as np
from math import *
def CELoss(z,y):
    '''
        y : True output. 
        z : Predicted output. 
        return : L
    '''
    L = 0
    for i in range(len(y)):
      L -= (y[i]*log10(z[i]) + (1-y[i])*log10(1-z[i]))
    return L
    #pass

### **0-1 Loss**
Loss Function used by perceptron is: $ \begin{cases} 
      0=z-y & z=y \\
      1=\dfrac{z-y}{z-y} & z\neq y
   \end{cases} $.

In [9]:
import numpy as np
def zeroOneLoss(z,y):
    '''
        y : True output. 
        z : Predicted output. 
        return : L
    '''
    sum = 0
    if np.array_equal(z,y):
      return 0
    else:
      return 1
    



## Cost Function
The cost function $J$ is commonly used to assess the performance of a model, and is defined with the loss function $L$ as follows:
$$\boxed{J(\theta)=\frac{1}{m}\sum_{i=1}^mL(h_\theta(x^{(i)}), y^{(i)})}$$
where $h_\theta$ is the hypothesis function i.e. the function used to predict the output.

In [11]:
lossFunctions = {
    "l1" : L1loss,
    "l2" : L2loss,
    "hinge" : hingeLoss,
    "cross-entropy" : CELoss,
    "0-1" : zeroOneLoss
}

def cost(Z : np.ndarray, Y : np.ndarray, loss : str):
    '''
        Z : a numpy array of predictions.
        Y : a numpy array of true values.
        return : A numpy array of costs calculated for each example.
    '''
    loss_func = lossFunctions[loss]
    A=[]
    for i in range(len()):  # should apply the loss function to each element of the arrays and return the array J
      A.append(loss_func(Z[i],Y[i]))
    return A

## Gradient Descent and Back Propagation
Gradient Descent is an algorithm that minimizes the loss function by calculating it's gradient. By noting $\alpha\in\mathbb{R}$ the learning rate, the update rule for gradient descent is expressed with the learning rate $\alpha$ and the cost function $J$ as follows:

$$\boxed{ W \longleftarrow W -\alpha\nabla J( W )}$$
​


But we need to find the partial derivative of Loss function wrt every parameter to know what is the slight change that we need to apply to our parameters. This becomes particularly hard if we have more than 1 layer in our algorithm. Here's where **Back Propagation** comes in. It's a way to find gradients wrt every parameter using the chain rule. Backpropagation is a method to update the weights in the neural network by taking into account the actual output and the desired output. The derivative with respect to weight ww is computed using chain rule and is of the following form:

$$\boxed{\frac{\partial L(z,y)}{\partial w}=\frac{\partial L(z,y)}{\partial a}\times\frac{\partial a}{\partial z}\times\frac{\partial z}{\partial w}}$$
​
 
As a result, the weight is updated as follows:

$$\boxed{w\longleftarrow w-\alpha\frac{\partial L(z,y)}{\partial w}}$$

So, In a neural network, weights are updated as follows:

* Step 1: Take a batch of training data.
* Step 2: Perform forward propagation to obtain the corresponding loss.
* Step 3: Backpropagate the loss to get the gradients.
* Step 4: Use the gradients to update the weights of the network.
​

Bonus Problem
 
Now, Assuming that you know Back Propagation (read a bit about it, if you don't), we'll now implement an image classification model on CIFAR-10.

# **Bonus Problem**

Now, Assuming that you know Back Propagation (read a bit about it, if you don't), we'll now implement an image classification model on CIFAR-10.

In [12]:
import tensorflow as tf  
 
# Display the version
print(tf.__version__)    
 
# other imports
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout
from tensorflow.keras.layers import GlobalMaxPooling2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model

2.8.0


In [13]:
# Load in the data
cifar10 = tf.keras.datasets.cifar10
 
# Distribute it to train and test set
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

# Reduce pixel values
x_train, x_test = x_train / 255.0, x_test / 255.0
 
# flatten the label values
y_train, y_test = y_train.flatten(), y_test.flatten()

170508288/170498071 [==============================] - 11s 0us/step
(50000, 32, 32, 3) (50000, 1) (10000, 32, 32, 3) (10000, 1)


In [ ]:
'''visualize data by plotting images'''
# YOUR CODE HERE
pass
# YOUR CODE HERE

In [ ]:

# number of classes
K = len(set(y_train))
'''
 calculate total number of classes
 for output layer
'''
print("number of classes:", K)
''' 
 Build the model using the functional API
 input layer
'''
```
  YOUR CODE HERE
```
 
'''Hidden layer'''
# YOUR CODE HERE
pass
# YOUR CODE HERE
 
"""last hidden layer i.e.. output layer"""
# YOUR CODE HERE
pass
# YOUR CODE HERE
 
 '''model description'''
model.summary()

In [ ]:
# Compile
...
  YOUR CODE HERE
...

In [ ]:
# Fit
...
  YOUR CODE HERE
...

In [ ]:
# label mapping
 
labels = '''airplane automobile bird cat deerdog frog horseship truck'''.split()
 
# select the image from our test dataset
image_number = 0
 
# display the image
plt.imshow(x_test[image_number])
 
# load the image in an array
n = np.array(x_test[image_number])
 
# reshape it
p = n.reshape(1, 32, 32, 3)
 
# pass in the network for prediction and
# save the predicted label
predicted_label = labels[model.predict(p).argmax()]
 
# load the original label
original_label = labels[y_test[image_number]]
 
# display the result
print("Original label is {} and predicted label is {}".format(
    original_label, predicted_label))